In [2]:
!pip install requests
    


In [ ]:
import os
import requests
import csv
from datetime import datetime, timezone
import time

def fetch_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data")
        return None

def add_timestamp_to_data(data, timestamp):
    if data:
        for subarray_index, subarray in enumerate(data):
            for entry in subarray:
                entry['timestamp'] = timestamp
                entry['subarray'] = subarray_index
        return data
    else:
        return None

def save_to_csv(data, file_path):
    if data:
        with open(file_path, 'w', newline='') as csvfile:
            fieldnames = ['x', 'y', 'timestamp', 'subarray']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for subarray in data:
                writer.writerows(subarray)

def check_integrity(data, file_path):
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        csv_data = list(csv_reader)
        original_row_count = sum(len(subarray) for subarray in data)
        csv_row_count = len(csv_data)
        if original_row_count == csv_row_count:
            print(f"CSV file '{file_path}' integrity check passed.")
        else:
            print(f"CSV file '{file_path}' integrity check failed.")

api_url = 'https://energie.theoxygent.nl/api/prices_v2.php'

# Create subfolder if it doesn't exist
subfolder = 'oxygent_data'
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

while True:
    current_time_gmt = datetime.now(timezone.utc).isoformat()
    file_timestamp = datetime.now(timezone.utc).strftime("%Y%m%d %H:%M")
    time_series_data = fetch_data_from_api(api_url) 
    time_series_data_with_timestamp = add_timestamp_to_data(time_series_data, current_time_gmt)
    csv_file_path = os.path.join(subfolder, f'time_series_data_{file_timestamp}.csv')
    save_to_csv(time_series_data_with_timestamp, csv_file_path)
    check_integrity(time_series_data_with_timestamp, csv_file_path)
    # Fetch data every hour
    time.sleep(3600)


CSV file 'oxygent_data/time_series_data_20250316 18:13.csv' integrity check passed.


In [ ]:
# code running on home assistant Addon jupyter lab
# import os
import requests
import csv
from datetime import datetime, timezone
import time

def fetch_data_from_api(url):
    """Fetch data from API with error handling."""
    try:
        response = requests.get(url, timeout=10)  # Timeout set to 10 seconds
        response.raise_for_status()  # Raise error if status code is not 200
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return None

def add_timestamp_to_data(data, timestamp):
    """Append timestamps to each data entry."""
    if data:
        for subarray_index, subarray in enumerate(data):
            for entry in subarray:
                entry['timestamp'] = timestamp
                entry['subarray'] = subarray_index
        return data
    return None

def save_to_csv(data, file_path):
    """Save the data into a CSV file."""
    if data:
        with open(file_path, 'w', newline='') as csvfile:
            fieldnames = ['x', 'y', 'timestamp', 'subarray']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for subarray in data:
                writer.writerows(subarray)

def check_integrity(data, file_path):
    """Verify that the CSV file correctly saved the data."""
    try:
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            csv_data = list(csv_reader)
        original_row_count = sum(len(subarray) for subarray in data)
        csv_row_count = len(csv_data)
        if original_row_count == csv_row_count:
            print(f"CSV file '{file_path}' integrity check passed.")
        else:
            print(f"CSV file '{file_path}' integrity check failed.")
    except Exception as e:
        print(f"Integrity check failed: {e}")

# API URL
api_url = 'https://energie.theoxygent.nl/api/prices_v2.php'

# Create subfolder if it doesn't exist
subfolder = 'oxygent_data'
os.makedirs(subfolder, exist_ok=True)

# Start hourly data fetching
while True:
    current_time_gmt = datetime.now(timezone.utc).isoformat()
    file_timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M")  # Safer filename format

    print(f"Fetching data at {file_timestamp} UTC...")
    
    time_series_data = fetch_data_from_api(api_url)
    
    if time_series_data:  # Only process if API returned valid data
        time_series_data_with_timestamp = add_timestamp_to_data(time_series_data, current_time_gmt)
        csv_file_path = os.path.join(subfolder, f'time_series_data_{file_timestamp}.csv')
        
        save_to_csv(time_series_data_with_timestamp, csv_file_path)
        check_integrity(time_series_data_with_timestamp, csv_file_path)
    
    # Wait exactly until the next hour to fetch again
    print("Sleeping for 1 hour...\n")
    time.sleep(3600)

NameError: name 'os' is not defined